In [1]:
newtype Parser s r = Parser { unParser :: ([s] -> ParseResult s r) }

newtype ParseResult s r = ParseResult { unParseResult :: [([s], r)] }

In [2]:
symbol :: Eq s => s -> Parser s s
symbol sym = Parser p
    where p (s:ss) | s == sym = ParseResult [(ss, sym)]
          p _                 = ParseResult []

In [3]:
satisfy :: (s -> Bool) -> Parser s s
satisfy pred = Parser p
    where p (s:ss) | pred s = ParseResult [(ss, s)]
          p _               = ParseResult []

In [4]:
{-# LANGUAGE InstanceSigs #-}

import Control.Applicative (Alternative(..))
import Control.Monad (ap)

In [5]:
instance Functor (Parser s) where
    fmap :: (r -> t) -> Parser s r -> Parser s t
    fmap f p = pure . f =<< p

instance Applicative (Parser s) where
    pure :: r -> Parser s r
    pure r = Parser $ \ss -> ParseResult [(ss,r)]

    (<*>) :: Parser s (r -> t) -> Parser s r -> Parser s t
    (<*>) = ap

instance Alternative (Parser s) where
    empty :: Parser s r
    empty = Parser $ \_ -> ParseResult []

    (<|>) :: Parser s r -> Parser s r -> Parser s r
    (<|>) (Parser p1) (Parser p2) = Parser $ \ss ->
        ParseResult $ unParseResult (p1 ss) ++ unParseResult (p2 ss)

instance Monad (Parser s) where
    (>>=) :: Parser s r -> (r -> Parser s t) -> Parser s t
    (>>=) (Parser p1) f = Parser $ \ss -> ParseResult
        [ tuple
        | (ssRest,result1) <- unParseResult $ p1 ss
        , tuple            <- unParseResult $ (unParser $ f result1) ssRest
        ]

In [6]:
aANDbORc :: Parser Char (Char, Char)
aANDbORc = do
    x <- symbol 'a'
    (\y -> (x,y)) <$> (symbol 'b' <|> symbol 'c')

unParseResult $ unParser aANDbORc "abc"
unParseResult $ unParser aANDbORc "acb"
unParseResult $ unParser aANDbORc "cba"

[("c",('a','b'))]

[("b",('a','c'))]

[]

In [7]:
alphaORhex :: Parser Char Char
alphaORhex = alpha <|> hex

alpha :: Parser Char Char
alpha = satisfy (\c -> elem c (['a'..'z']++['A'..'Z']))

hex :: Parser Char Char
hex = satisfy (\c -> elem c (['0'..'9']++['A'..'F']))

unParseResult $ unParser alphaORhex "abc"
unParseResult $ unParser alphaORhex "ABC"
unParseResult $ unParser alphaORhex "123"

[("bc",'a')]

[("BC",'A'),("BC",'A')]

[("23",'1')]

In [8]:
import Data.Char

word :: Parser Char String
word = some (satisfy isAlpha)

sep :: Parser Char String
sep = some (satisfy isSpace <|> symbol ',')

sentence :: Parser Char [String]
sentence = do
    w <- word
    r <- many (sep >>= \_ -> word)
    (\_ -> (w:r)) <$> symbol '.'

unParseResult $ unParser word "Hello world"

[(" world","Hello"),("o world","Hell"),("lo world","Hel"),("llo world","He"),("ello world","H")]

In [9]:
newtype ParserC s t r = ParserC
    { unParserC :: Success s t r -> NextRes s t -> Parser s t }
newtype Success s t r = Success
    { unSuccess :: r -> NextRes s t -> Parser s t }
type NextRes s t = ParseResult s t

In [10]:
instance Functor (ParserC s t) where
    fmap :: (r -> u) -> ParserC s t r -> ParserC s t u
    fmap f p = pure . f =<< p
    
instance Applicative (ParserC s t) where
    pure :: r -> ParserC s t r
    pure r = ParserC $ \(Success succ) next -> succ r next
    
    (<*>) :: ParserC s t (u -> v) -> ParserC s t u -> ParserC s t v
    (<*>) = ap

instance Alternative (ParserC s t) where
    empty :: ParserC s t r
    empty = ParserC $ \succ next -> Parser $ \ss -> next
    
    (<|>) :: ParserC s t r -> ParserC s t r -> ParserC s t r
    (<|>) (ParserC p1) (ParserC p2) =
        ParserC $ \sc nc -> Parser $ \ss -> 
            unParser (p1 sc (unParser (p2 sc nc) ss)) ss

instance Monad (ParserC s t) where
    (>>=) :: ParserC s t u -> (u -> ParserC s t v) -> ParserC s t v
    (>>=) (ParserC p1) f =
        ParserC $ \sc -> p1 $ Success $ \t -> unParserC (f t) sc

begin :: ParserC s t t -> Parser s t
begin (ParserC p) = p
    (Success $ \r (ParseResult nc) -> Parser $ \ss -> ParseResult ((ss,r):nc))
    (ParseResult [])

In [11]:
symbolC :: Eq s => s -> ParserC s t s
symbolC sym = ParserC $ \(Success sc) nc -> Parser $ \ss -> case ss of
    (s:ss) | s == sym -> unParser (sc sym nc) ss
    _                 -> nc

satisfyC :: (s -> Bool) -> ParserC s t s
satisfyC pred = ParserC $ \(Success sc) nc -> Parser $ \ss -> case ss of
    (s:ss) | pred s -> unParser (sc s nc) ss
    _               -> nc

wordC :: ParserC Char t String
wordC = some (satisfyC isAlpha)

sepC :: ParserC Char t String
sepC = some (satisfyC isSpace <|> symbolC ',')

sentenceC :: ParserC Char t [String]
sentenceC = do
    w <- wordC
    r <- many (sepC >>= \_ -> wordC)
    (\_ -> (w:r)) <$> symbolC '.'

unParseResult $ unParser (begin wordC) "Hello world"

[(" world","Hello"),("o world","Hell"),("lo world","Hel"),("llo world","He"),("ello world","H")]

In [23]:
import System.TimeIt


timeIt $ print . length . (unParseResult . unParser sentence) =<< readFile "loremipsum.txt"

0
CPU time:  82.99s

In [22]:
timeIt $ print . length . (unParseResult . unParser (begin sentenceC)) =<< readFile "loremipsum.txt"

0
CPU time:   5.80s